In [0]:
alerts_df = spark.read.table("default.real_time_alerts")
profiles_df = spark.read.table("default.user_behavior_profiles")

In [0]:
print("Iniciando testes de qualidade...")
# TESTE 1: A chave primária da tabela de perfis (user_id) deve ser única.
assert profiles_df.select("user_id").distinct().count() == profiles_df.count(), "ERRO DE QUALIDADE: user_id não é único na tabela de perfis."
print(" Passou no teste de uid único")

# TESTE 2: Os valores de alerta devem ser sempre positivos.
assert alerts_df.filter("amount_brl <= 0").count() == 0, "ERRO DE QUALIDADE: Encontrado alerta com valor negativo ou zero."
print(" Passou no teste de valores negativos")

# TESTE 3: Nenhum perfil de usuário deve ter um desvio padrão negativo.
assert profiles_df.filter("stddev_amount < 0").count() == 0, "ERRO DE QUALIDADE: Encontrado desvio padrão negativo nos perfis."
print("Passou no teste de desvio padrão negativo")